# Ejercicio 5

Genere un video en un patio o en un hall de edificio donde en un principio se vea vacío y luego aparezca una persona. Mediante los métodos de motion detection (sin usar deep learning) logre una detección de la persona cuando entra al cuadro suponiendo la utilidad para una cámara de seguridad. 
Luego sobre el mismo video aplique los algoritmos de flujo denso y disperso que se mostraron en clase. 
Escriba una reflexión sobre los resultados en el formato md dentro del Jupyter Notebook.

In [1]:
import cv2
import numpy as np

# Función actualizada para realizar diferencia de fotogramas con normalización:
def process_frame_difference(new_image, prev_image, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # Convertir la imagen umbralizada a color para mantener la consistencia con el video original
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

    return thresh_color



In [2]:
video_capture = cv2.VideoCapture('../fotos/video2.mp4')

# Lee el primer fotograma
prev_frame = None
ret, prev_frame = video_capture.read()

# Itera sobre los fotogramas restantes
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # Procesa la diferencia entre fotogramas
    processed_frame = process_frame_difference(frame, prev_frame)

    # Muestra el fotograma procesado
    cv2.imshow('Processed Frame', processed_frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Actualiza el fotograma anterior
    prev_frame = frame.copy()

# Libera los recursos y cierra las ventanas
video_capture.release()
cv2.destroyAllWindows()

In [3]:
# Función para procesar el flujo óptico denso
def process_dense_optical_flow(new_image, prev_image):
    # Convierte la nueva imagen a escala de grises
    gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    if not hasattr(process_dense_optical_flow, "init_done"):
        process_dense_optical_flow.prev_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        process_dense_optical_flow.mask = np.zeros_like(new_image)
        process_dense_optical_flow.mask[..., 1] = 255
        process_dense_optical_flow.init_done = True

    if process_dense_optical_flow.init_done:
        prev_gray = process_dense_optical_flow.prev_gray
        mask = process_dense_optical_flow.mask

    # Calcula el flujo óptico
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Computa magnitud y ángulo de los vectores 2D
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Establece el tono de la imagen según la dirección del flujo óptico
    mask[..., 0] = angle * 180 / np.pi / 2
    # Establece el valor de la imagen según la magnitud del flujo óptico
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convierte de HSV a RGB
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    # Actualiza la imagen previa a gris
    process_dense_optical_flow.prev_grayprev_gray = gray.copy()
    return rgb

In [4]:
video_capture = cv2.VideoCapture('../fotos/video2.mp4')

# Lee el primer fotograma
prev_frame = None
ret, prev_frame = video_capture.read()

# Itera sobre los fotogramas restantes
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # Procesa la diferencia entre fotogramas
    processed_frame = process_dense_optical_flow(frame, prev_frame)

    # Muestra el fotograma procesado
    cv2.imshow('Processed Frame', processed_frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Actualiza el fotograma anterior
    prev_frame = frame.copy()

# Libera los recursos y cierra las ventanas
video_capture.release()
cv2.destroyAllWindows()

In [5]:
def process_sparse_optical_flow(new_image, prev_image):
    # Preparamos las imagenes de trabajo
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    prev_gray_image = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)

    # Verificar si ya se han detectado las características de Shi-Tomasi
    if not hasattr(process_sparse_optical_flow, "shi_tomasi_done"):
        # Definir parámetros para la detección de esquinas de Shi-Tomasi
        feature_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)
        # Detectar puntos característicos en la imagen
        process_sparse_optical_flow.prev_points = cv2.goodFeaturesToTrack(new_gray, mask=None, **feature_params)
        # Crear una máscara para dibujar el flujo óptico
        process_sparse_optical_flow.mask = np.zeros_like(new_image)
        # Marcar que se ha completado la detección de Shi-Tomasi
        process_sparse_optical_flow.shi_tomasi_done = True

    # Continuar si se ha completado la detección de Shi-Tomasi
    if process_sparse_optical_flow.shi_tomasi_done:
        prev_points = process_sparse_optical_flow.prev_points
        mask = process_sparse_optical_flow.mask

    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calcular el flujo óptico de Lucas-Kanade
    new_points, status, error = cv2.calcOpticalFlowPyrLK(prev_gray_image, new_gray, prev_points, None, **lk_params)
    # Filtrar puntos buenos
    good_old = prev_points[status == 1]
    good_new = new_points[status == 1]
    color = (0, 255, 0)  # Color para el dibujo
    # Dibujar el movimiento (flujo óptico)
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.astype(int).ravel()
        c, d = old.astype(int).ravel()
        mask = cv2.line(mask, (a, b), (c, d), color, 2)
        new_image = cv2.circle(new_image, (a, b), 3, color, -1)

    # Combinar la imagen actual con las líneas de flujo óptico dibujadas
    output = cv2.add(new_image, mask)
    # Actualizar puntos para el siguiente cuadro
    process_sparse_optical_flow.prev_points = good_new.reshape(-1, 1, 2)
    return output

In [6]:
video_capture = cv2.VideoCapture('../fotos/video2.mp4')

# Lee el primer fotograma
prev_frame = None
ret, prev_frame = video_capture.read()

# Itera sobre los fotogramas restantes
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # Procesa la diferencia entre fotogramas
    processed_frame = process_sparse_optical_flow(frame, prev_frame)

    # Muestra el fotograma procesado
    cv2.imshow('Processed Frame', processed_frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Actualiza el fotograma anterior
    prev_frame = frame.copy()

# Libera los recursos y cierra las ventanas 
video_capture.release()
cv2.destroyAllWindows()

# Reflexión:

Los algoritmos aplicados al video grabado dentro de una casa revelaron resultados satisfactorios en cuanto a su utilidad como detectores de movimientos.

El algoritmo de flujo óptico denso al comienzo pareciera que no detecta nada pero resultó eficaz al identificar claramente la presencia de una silueta en movimiento. Este método logro detectar cambios significativos y delinear la forma en movimiento.

Por otro lado, el algoritmo de flujo disperso, realizó el seguimiento del movimiento de la persona, proporcionando líneas que delinean el trayecto seguido por la persona en movimiento.

Es esencial tener en cuenta diversos factores, como la calidad de la cámara, su posición y las condiciones de iluminación, para garantizar un enfoque óptimo en la detección de movimientos. Estos aspectos tienen un impacto significativo en la precisión y fiabilidad de los resultados obtenidos, lo que asegura una detección de movimientos precisa y confiable.
